# Worksheet 08

Name:  Ashton Fox
UID: U95686268

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.971263591595482, 5.4287981496701905, 4.74461980784747, 5.921373727448982, 3.645173079537715, 5.198457204912151, 4.415920041961826, 4.545471125639444, 2.517945154148651, 5.922548718881171]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
mean = 8
var = 1
s2 = np.random.normal( mean , var**2 , 10 ).tolist()
print(s2)

[6.089138065805974, 7.875133762888543, 8.5789195338976, 9.062925985991638, 6.7285813095114335, 6.639478394815296, 7.317085268302237, 7.517137490062556, 8.004031365141765, 7.836424813736746]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.922548718881171, 2.517945154148651, 4.545471125639444, 7.836424813736746, 8.004031365141765, 4.415920041961826, 7.517137490062556, 7.317085268302237, 5.198457204912151, 3.645173079537715]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

prior probability (1/2) of selecting data from either component, mean and variance

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2)/(len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x: (x-mean[1])**2, s2)) /len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[2.517945154148651, 4.545471125639444, 4.415920041961826, 5.198457204912151, 3.645173079537715]
[5.922548718881171, 7.836424813736746, 8.004031365141765, 7.517137490062556, 7.317085268302237]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.064593321239957,  mean_2 = 7.319445531224895
var_1 = 0.8416710252139813,  var_2 = 0.5452667456624631


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.922548718881171
probability of observing that point if it came from cluster 0 =  0.04146054461128171
probability of observing that point if it came from cluster 1 =  0.027487593728191474
point =  2.517945154148651
probability of observing that point if it came from cluster 0 =  0.0876025514646408
probability of observing that point if it came from cluster 1 =  1.0624582483142964e-17
point =  4.545471125639444
probability of observing that point if it came from cluster 0 =  0.40261058541388206
probability of observing that point if it came from cluster 1 =  1.754841773383634e-06
point =  7.836424813736746
probability of observing that point if it came from cluster 0 =  2.0648465554671758e-05
probability of observing that point if it came from cluster 1 =  0.4667666456339912
point =  8.004031365141765
probability of observing that point if it came from cluster 0 =  8.293032335097626e-06
probability of observing that point if it came from cluster 1 =  0.33266929040497695
point 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [8]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), ... ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), 
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.264059400985733,  mean_2 = 7.509378235532319
var_1 = 1.0831475288431125,  var_2 = 0.31888843980603676


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [12]:
prob_s0_x = [] # P(C_0 | X_i)
prob_s1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_s1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  5.922548718881171
probability of observing that point if it came from cluster 0 =  0.11405671805579913
probability of observing that point if it came from cluster 1 =  5.251762922035689e-06


TypeError: unsupported operand type(s) for *: 'ellipsis' and 'float'

probability becomes a bit more certain than in c1

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [10]:
assignments = [[], []]
for x in zip(data, prob_s0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [5.922548718881171, 2.517945154148651, 4.545471125639444, 4.415920041961826, 5.198457204912151, 3.645173079537715]
points in cluster 1 : [7.836424813736746, 8.004031365141765, 7.517137490062556, 7.317085268302237]
